In [ ]:
import os
import json

from dotenv import load_dotenv
from docling.document_converter import DocumentConverter

from schemas.pydantic_schema import DispositifAide

/Users/theo.moreau/Documents/betagouv/aides-agri/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()
API_KEY = os.getenv("ALBERT_API_KEY")
ENDPOINT = "https://albert.api.etalab.gouv.fr/v1/agents/completions"

In [3]:
document_path = [
  "https://aides-redevances.eau-loire-bretagne.fr/files/live/sites/aides-redevances/files/Aides-12prog/Fiches-actions/AGRI_1.pdf",
  "./2017-561_final.pdf"
]

In [4]:
def get_document_context(file_path):
  converter = DocumentConverter()
  doc = converter.convert(file_path).document
  return doc.export_to_text()

In [5]:
INSTRUCTION_PROMPT = """
You are an assistant that extracts information from only the context into a structured format.

Your task:
- Return ONLY a valid JSON object that strictly follows the schema shown below.
- Do NOT add, invent, or hallucinate information. If some fields are missing in the context, use null.
- Do NOT include explanations, extra text, or comments — JSON only.
- The keys, types, and structure must exactly match the schema.

Context: 
"""

In [6]:
prompt = INSTRUCTION_PROMPT + get_document_context(document_path[0])

2025-09-18 09:35:11,535 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-09-18 09:35:11,915 - INFO - Going to convert document batch...
2025-09-18 09:35:11,915 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-09-18 09:35:11,984 - INFO - Loading plugin 'docling_defaults'
2025-09-18 09:35:11,991 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-09-18 09:35:12,009 - INFO - Loading plugin 'docling_defaults'
2025-09-18 09:35:12,029 - INFO - Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-09-18 09:35:13,233 - INFO - Accelerator device: 'mps'
2025-09-18 09:35:15,158 - INFO - Accelerator device: 'mps'
2025-09-18 09:35:16,639 - INFO - Accelerator device: 'mps'
2025-09-18 09:35:17,174 - INFO - Processing document AGRI_1.pdf
2025-09-18 09:35:22,539 - INFO - Finished converting document AGRI_1.pdf in 11.58 sec.
2025-09-18 09:35:22,649 - WARNING - Parameter `strict_text` has 

In [16]:
from ollama import chat

response = chat(
  messages=[
    {
      'role': 'user',
      'content': prompt,
    }
  ],
  model='gemma3:4b',
  format=DispositifAide.model_json_schema(),
)

structured_output = DispositifAide.model_validate_json(response.message.content)
print(structured_output)

2025-09-18 10:01:28,827 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


titre='Accompagnement collectif et individuel des agriculteurs' description="Favoriser la mobilisation des agriculteurs vers des changements de pratiques et de systèmes efficaces et durables, par l'accompagnement collectif et/ou individuel (diagnostic d'exploitation et conseil individuel)." eligibilite=['Public ou privé.'] types_aides=['financement'] porteurs=[Porteur(nom="Agence de l'eau", roles=['financeur', 'diffuseur', 'diffuseur'])] programmes_parents=None url_source=None cibles=["aires d'alimentation de captages prioritaires inscrites au Sdage en vigueur", "masses d'eau des 22 plans d'eau prioritaires sensibles à l'eutrophisation pour le phosphore", 'bassins versants algues vertes', "sous-bassins en déséquilibre quantitatif des projets de territoire pour la gestion de l'eau (PTGE) approuvés", "masses d'eau en risque de non atteinte du bon état pour les paramètres pesticides et/ou nitrates et proches du bon état"] eligibilite_geographique='sur les territoires suivants, identifiés 

In [19]:
dict(structured_output)

{'titre': 'Accompagnement collectif et individuel des agriculteurs',
 'description': "Favoriser la mobilisation des agriculteurs vers des changements de pratiques et de systèmes efficaces et durables, par l'accompagnement collectif et/ou individuel (diagnostic d'exploitation et conseil individuel).",
 'eligibilite': ['Public ou privé.'],
 'types_aides': ['financement'],
 'porteurs': [Porteur(nom="Agence de l'eau", roles=['financeur', 'diffuseur', 'diffuseur'])],
 'programmes_parents': None,
 'url_source': None,
 'cibles': ["aires d'alimentation de captages prioritaires inscrites au Sdage en vigueur",
  "masses d'eau des 22 plans d'eau prioritaires sensibles à l'eutrophisation pour le phosphore",
  'bassins versants algues vertes',
  "sous-bassins en déséquilibre quantitatif des projets de territoire pour la gestion de l'eau (PTGE) approuvés",
  "masses d'eau en risque de non atteinte du bon état pour les paramètres pesticides et/ou nitrates et proches du bon état"],
 'eligibilite_geogr